In [1]:
from collections import defaultdict
import os, glob, random, sys, time, keras, cv2, itertools, sklearn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from keras.preprocessing import image
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Model
from collections import Counter
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.utils.fixes import signature
from keras.utils import to_categorical
from keras.datasets import fashion_mnist
from keras.models import *
from keras.layers import *
from skimage import io
from sklearn.model_selection import train_test_split

C:\Users\kamru\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
CurrentDir=os.getcwd()
Dataset = os.listdir(CurrentDir+'/yalefacesData/')

imageData = []
imageLabels = []

for file in Dataset:
    if file!='Readme.txt':
        fileRead=CurrentDir+'/yalefacesData/'+file
#         print(fileRead)       
        imgRead = io.imread(fileRead, as_grey=True)
        imageData.append(imgRead)    
        labelRead = int(os.path.split(file)[1].split(".")[0].replace("subject", "")) - 1 # Parse class label from file
        imageLabels.append(labelRead)
print(len(imageData))
print(imageData[0].shape)
print(imageData[0].dtype)

print(len(imageLabels))
print(imageLabels)




# Save the images in Test folder directory 
for i in range(len(imageData)):
    image=imageData[i]
    number='%03d'%i
    path=CurrentDir+'/test/'+str(number)+'.png'
    cv2.imwrite(path,image)

166
(243, 320)
uint8
166
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]


In [3]:
faceDetectClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [10]:
imageDataFin = []
for i in imageData:
    facePoints = faceDetectClassifier.detectMultiScale(i)
#     print(facePoints[0])
    x,y = facePoints[0][:2]
    cropped = i[y: y + 150, x: x + 150]
    imageDataFin.append(cropped)

In [7]:
imageDataFin[0].shape

(150, 150)

In [11]:
c = np.array(imageDataFin)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(np.array(imageDataFin),
                                                    np.array(imageLabels), 
                                                    train_size=0.9, 
                                                    random_state = 20)

C:\Users\kamru\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [20]:
X_train = np.array(X_train)
X_test = np.array(X_test)
print(X_train.shape)
print(X_test.shape)
nb_classes = 15
y_train = np.array(y_train) 
y_test = np.array(y_test)
Y_train = to_categorical(y_train, nb_classes)
Y_test = to_categorical(y_test, nb_classes)

(149, 150, 150)
(17, 150, 150)


In [23]:
X_train = X_train.reshape(149, 150*150)
X_test = X_test.reshape(17, 150*150)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Training matrix shape (149, 22500)
Testing matrix shape (17, 22500)


In [25]:
model = Sequential()
model.add(Dense(512,input_shape=(X_train.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               11520512  
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 15)                7695      
__________

In [26]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [27]:
model.fit(X_train, Y_train, batch_size=64, nb_epoch=50, verbose=1, validation_data=(X_test, Y_test))

C:\Users\kamru\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 149 samples, validate on 17 samples
Epoch 1/50
149/149 [==============================] - 5s 30ms/step - loss: 2.7082 - acc: 0.0336 - val_loss: 2.7119 - val_acc: 0.0000e+00
Epoch 2/50
149/149 [==============================] - 0s 468us/step - loss: 2.7080 - acc: 0.0940 - val_loss: 2.7154 - val_acc: 0.0000e+00
Epoch 3/50
149/149 [==============================] - 0s 471us/step - loss: 2.7071 - acc: 0.0805 - val_loss: 2.7190 - val_acc: 0.0000e+00
Epoch 4/50
149/149 [==============================] - 0s 444us/step - loss: 2.7067 - acc: 0.0604 - val_loss: 2.7230 - val_acc: 0.0000e+00
Epoch 5/50
149/149 [==============================] - 0s 437us/step - loss: 2.7062 - acc: 0.0671 - val_loss: 2.7276 - val_acc: 0.0000e+00
Epoch 6/50
149/149 [==============================] - 0s 447us/step - loss: 2.7056 - acc: 0.1007 - val_loss: 2.7325 - val_acc: 0.0000e+00
Epoch 7/50
149/149 [==============================] - 0s 451us/step - loss: 2.7061 - acc: 0.0872 - val_loss: 2.7369 - val_acc: 0

In [31]:
score = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

17/17 [==============================] - 0s 176us/step
Test loss: 2.791778326034546
Test accuracy: 0.0
